## Mission to Mars

In [2]:
# import and setup dependencies

# to scrape from html
from bs4 import BeautifulSoup as bs

# to control browser
from splinter import Browser
browser = Browser("chrome", headless=False)

# to format data
import pandas as pd

# to format URLs
from urllib.parse import urlsplit

# for pause
import time

### NASA Mars News

In [3]:
# set url to NASA news site and visit with splinter
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
# get html from splinter and give to BeautifulSoup
html = browser.html
page_to_parse = bs(html,"html.parser")

In [5]:
# use soup to pull first article title and paragraph
news_title = page_to_parse.find("div",class_="content_title").find("a").text
news_p = page_to_parse.find("div", class_="article_teaser_body").text

# print to check
print(news_title)
print(news_p)

What's Mars Solar Conjunction, and Why Does It Matter?
NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.


### JPL Mars Space Images - Featured Image

In [6]:
# set link to get jpl image, send splinter to fetch
jpl_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_image_url)

In [7]:
# navigate browser to full size image
browser.click_link_by_partial_text('FULL IMAGE')
# give page time to load
time.sleep(5)
browser.click_link_by_partial_text('more info')

In [8]:
# get html from splinter and give to BeautifulSoup
html = browser.html
page_to_parse = bs(html,"html.parser")

In [9]:
# find image url with soup
img_src = page_to_parse.find('img', class_='main_image').get('src')

# get the base url
base_url = "{0.scheme}://{0.netloc}".format(urlsplit(jpl_image_url))

# put them together
featured_image_url=base_url + img_src

# print to check
print(base_url)
print(img_src)
print(featured_image_url)

https://www.jpl.nasa.gov
/spaceimages/images/largesize/PIA19113_hires.jpg
https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19113_hires.jpg


### Mars Weather

In [10]:
#get weather tweet
tweet_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(tweet_url)

In [11]:
# xpath to tweet div
tweet_xpath='//*[@id="stream-item-tweet-1164580766023606272"]/div[1]/div[2]/div[2]/p'
mars_weather  = browser.find_by_xpath(tweet_xpath).text

# print to check
print(mars_weather)

InSight sol 259 (2019-08-19) low -101.0ºC (-149.8ºF) high -27.1ºC (-16.9ºF)
winds from the SW at 4.3 m/s (9.6 mph) gusting to 17.6 m/s (39.4 mph)
pressure at 7.60 hPa


### Mars Facts

In [12]:
# get mars facts
url = "https://space-facts.com/mars/"
mars_table = pd.read_html(url)

In [13]:
# import table into df, make df into table ... ok
mars_facts=mars_table[1]
mars_facts.set_index(0, inplace=True)
mars_facts = mars_facts.to_html()
mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>1</th>\n    </tr>\n    <tr>\n      <th>0</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n<

### Mars Hemispheres

In [14]:
# go to USGS Astrogeology get html
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
# get the base url
base_url = "{0.scheme}://{0.netloc}".format(urlsplit(url))

In [15]:
browser.visit(url)
html = browser.html
page_to_parse = bs(html,"html.parser")

In [16]:
# collect titles and links to hemisphere_image_urls
hemisphere_image_urls = []
hemisphere_image_urls_titles = []
hemisphere_image_urls_images = []
img_urls = []

# collect container divs with links and titles
item_div = page_to_parse.find_all('div', class_='item')

# iterate through containers
for item in item_div: 
# grab title
    title = item.find('h3').text
# collect links
    img_urls.append(item.find('a', class_='product-item')['href'])
    hemisphere_image_urls_titles.append(title)

# visit collected links to gather large image links
for link in img_urls:
    browser.visit(base_url + link)
    img_html = browser.html
    page_to_parse = bs(img_html, 'html.parser')
# collect links
    full_img_url = base_url + page_to_parse.find('img', class_='wide-image')['src']
    hemisphere_image_urls_images.append(full_img_url)
    
for url, title in zip(hemisphere_image_urls_images, hemisphere_image_urls_titles):
    hemisphere_image_urls.append({"title" : title, "img_url" : url})



In [19]:
# make dictionary out of all collected data for later use in flask app
mars_info={"news_title":news_title,
           "news_p":news_p,
           "featured_image_url":featured_image_url,
           "tweet_url":tweet_url,
           "mars_weather":mars_weather,
           "mars_facts":mars_facts,
           "hemisphere_image_urls":hemisphere_image_urls    
          }
print(mars_info)

{'news_title': "What's Mars Solar Conjunction, and Why Does It Matter?", 'news_p': 'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19113_hires.jpg', 'tweet_url': 'https://twitter.com/marswxreport?lang=en', 'mars_weather': 'InSight sol 259 (2019-08-19) low -101.0ºC (-149.8ºF) high -27.1ºC (-16.9ºF)\nwinds from the SW at 4.3 m/s (9.6 mph) gusting to 17.6 m/s (39.4 mph)\npressure at 7.60 hPa', 'mars_facts': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>1</th>\n    </tr>\n    <tr>\n      <th>0</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6